<a href="https://colab.research.google.com/github/krishnendu19802/MovieRecommendor/blob/main/MovieRecommendor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install scikit-surprise pandas


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 6.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357279 sha256=4472cd0b6fa46d68047c46e05186df5800333ac7cb3d6e568f196befe509f46d
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [6]:
import pandas as pd
import zipfile
import io
import requests
from surprise import Reader, Dataset, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise.model_selection import cross_validate

In [7]:


# URL for the MovieLens dataset
url = "https://files.grouplens.org/datasets/movielens/ml-latest-small.zip"

# Download the ZIP file
response = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(response.content))

# List files in the zip archive (to find the correct CSV file)
print(z.namelist())

# Extract and load the CSV file (the one we need is 'ratings.csv')
with z.open('ml-latest-small/ratings.csv') as file:
    movie_data = pd.read_csv(file, usecols=['userId', 'movieId', 'rating'])

# Verify data loaded correctly
print(movie_data.head())


['ml-latest-small/', 'ml-latest-small/links.csv', 'ml-latest-small/tags.csv', 'ml-latest-small/ratings.csv', 'ml-latest-small/README.txt', 'ml-latest-small/movies.csv']
   userId  movieId  rating
0       1        1     4.0
1       1        3     4.0
2       1        6     4.0
3       1       47     5.0
4       1       50     5.0


In [8]:



reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(movie_data[['userId', 'movieId', 'rating']], reader)

# Train-test split
trainset, testset = train_test_split(data, test_size=0.2)

# Use the SVD algorithm for collaborative filtering
model = SVD()

# Train the model
model.fit(trainset)

# Make predictions on the test set
predictions = model.test(testset)

# Calculate and print the accuracy
accuracy.rmse(predictions)

# Cross-validation (optional)
cross_validate(model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# Function to get movie recommendations for a specific user
def get_movie_recommendations(user_id, n=5):
    all_movie_ids = movie_data['movieId'].unique()
    watched_movie_ids = movie_data[movie_data['userId'] == user_id]['movieId'].unique()

    # Find movies the user hasn't watched
    unwatched_movie_ids = [movie_id for movie_id in all_movie_ids if movie_id not in watched_movie_ids]

    # Predict ratings for unwatched movies
    predictions = [model.predict(user_id, movie_id) for movie_id in unwatched_movie_ids]

    # Sort by predicted rating
    recommendations = sorted(predictions, key=lambda x: x.est, reverse=True)

    # Return top 'n' movie recommendations
    return recommendations[:n]

# Get top 5 movie recommendations for a user
user_id = 1  # Example user_id
recommendations = get_movie_recommendations(user_id, n=5)

# Print the recommendations
for rec in recommendations:
    print(f"MovieID: {rec.iid}, Predicted Rating: {rec.est:.2f}")


RMSE: 0.8733
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8720  0.8732  0.8739  0.8782  0.8759  0.8746  0.0022  
MAE (testset)     0.6708  0.6711  0.6712  0.6729  0.6733  0.6719  0.0010  
Fit time          1.31    1.53    1.33    1.33    1.35    1.37    0.08    
Test time         0.25    0.20    0.28    0.13    0.23    0.22    0.05    
MovieID: 318, Predicted Rating: 5.00
MovieID: 58559, Predicted Rating: 5.00
MovieID: 898, Predicted Rating: 5.00
MovieID: 899, Predicted Rating: 5.00
MovieID: 908, Predicted Rating: 5.00
